In [45]:
%%writefile code1.cpp
#include<iostream>
#include<climits>     /*Used for INT_MAX*/
using namespace std;
#define vertex 6     /*It is the total no of verteices in the graph*/
int minimumDist(int dist[], bool Dset[])   /*A method to find the vertex with minimum distance which is not yet included in Dset*/
{
	int min=INT_MAX,index;                 /*initialize min with the maximum possible value as infinity does not exist */
	for(int v=0;v<vertex;v++)
	{
		if(Dset[v]==false && dist[v]<=min)      
		{
			min=dist[v];
			index=v;
		}
	}
	return index;
}
void dijkstra(int graph[vertex][vertex],int src) /*Method to implement shortest path algorithm*/
{
	int dist[vertex];                             
	bool Dset[vertex];
	for(int i=0;i<vertex;i++)                    /*Initialize distance of all the vertex to INFINITY and Dset as false*/  
	{
		dist[i]=INT_MAX;
		Dset[i]=false;	
	}
	dist[src]=0;                                   /*Initialize the distance of the source vertec to zero*/
	for(int c=0;c<vertex;c++)                           
	{
		int u=minimumDist(dist,Dset);              /*u is any vertex that is not yet included in Dset and has minimum distance*/
		Dset[u]=true;                              /*If the vertex with minimum distance found include it to Dset*/ 
		for(int v=0;v<vertex;v++)                  
		/*Update dist[v] if not in Dset and their is a path from src to v through u that has distance minimum than current value of dist[v]*/
		{
			if(!Dset[v] && graph[u][v] && dist[u]!=INT_MAX && dist[u]+graph[u][v]<dist[v])
			dist[v]=dist[u]+graph[u][v];
		}
	}
	cout<<"Vertex\t\tDistance from source"<<endl;
	for(int i=0;i<vertex;i++)                       /*will print the vertex with their distance from the source to the console */
	{
		char c=65+i;
		cout<<c<<"\t\t"<<dist[i]<<endl;
	}
}
int main()
{
	
	int graph[vertex][vertex]={{0,40,15,0,0,0},{0,0,20,10,25,6},{0,0,0,100,0,0},{0,0,0,0,0,0},{0,0,0,0,0,8},{0,0,0,0,0,0}};
	dijkstra(graph,0);	
	return 0;	                        
}

Overwriting code1.cpp


In [46]:
!g++  code1.cpp

In [47]:
!./a.out

Vertex		Distance from source
A		0
B		40
C		15
D		50
E		65
F		46


In [48]:
%%writefile codes.cpp
#include <stdio.h>
#include <omp.h>
#include<algorithm>

#define INFINITY 100000
int V,E;

//Structure for vertex
typedef struct 
{
	int label;
	bool visited;	

} Vertex;

//Structure for directed edge from u to v
typedef struct
{
	int u;
	int v;

} Edge;

//Printing Shortest Path Length
void printShortestPathLength(int *path_length)
{
	printf("\nVERTEX \tSHORTEST PATH LENGTH \n");
	int i;
	for(i = 0; i < V; i++)
	{
		printf("%d \t",i);
		if (path_length[i]<INFINITY)
			printf("%d\n",path_length[i]);
		else
			printf("Infinity\n");
	}
}


//Finds weight of the edge that connects Vertex u with Vertex v
int findEdgeWeight(Vertex u, Vertex v, Edge *edges, int *weights)
{

	int i;
	for(i = 0; i < E; i++)
	{

		if(edges[i].u == u.label && edges[i].v == v.label)
		{
			return weights[i];
		}
	}
	// If no edge exists, weight is infinity
	return INFINITY;
}

//Get the minimum path length among the paths
int minimimPathLength(int *path_length, Vertex *vertices)
{
	int i;
	int min_path_length = INFINITY;
	for(i = 0; i < V; i++)
	{
		if(vertices[i].visited == true)
		{
			continue;
		}
		
		else if(vertices[i].visited == false && path_length[i] < min_path_length)
		{
			min_path_length = path_length[i];
			
		}
		
	}
	return min_path_length;
}

int minimimPathVertex(Vertex *vertices, int *path_length)
{
	int i;
	int min_path_length = minimimPathLength(path_length, vertices);
	
	//Get the vertex with the minimum path length
	//Mark it as visited
	for(i = 0; i < V; i++)
	{
		if(vertices[i].visited == false && path_length[vertices[i].label] == min_path_length)
		{
			vertices[i].visited = true;
			return i;
		}
	}
}

// Dijkstra Algorithm
void Dijkstra_Parallel(Vertex *vertices, Edge *edges, int *weights, Vertex *root)
{	

	double parallel_start, parallel_end;
	int path_length[V];

	// Mark first vertex as visited, shortest path = 0
	root->visited = true;
	path_length[root->label] = 0;
	int i, j;
	// Compute distance to other vertices
	for(i = 0; i < V;i++)
	{

		if(vertices[i].label != root->label)
		{
			path_length[vertices[i].label] = findEdgeWeight(*root, vertices[i], edges, weights);
		}
		else
		{
		
			vertices[i].visited = true;
		}
	}
	
	parallel_start = omp_get_wtime();
	// External For Loop
	for(j = 0; j < V; j++)
	{
		Vertex u;
		// Obtain the vertex which has shortest distance and mark it as visited
		int h = minimimPathVertex(vertices, path_length);
		u = vertices[h];	

		//Update shortest path wrt new source 
		//Internal For Loop, Parallelising the computation
		#pragma omp parallel for schedule(static) private(i)
		for(i = 0; i < V; i++)
		{
			if(vertices[i].visited == false)
			{	
				int c = findEdgeWeight( u, vertices[i], edges, weights);
				path_length[vertices[i].label] = std::min(path_length[vertices[i].label], path_length[u.label] + c);
			}
		}
	}
	parallel_end = omp_get_wtime();
	printShortestPathLength(path_length);
	printf("\nRunning time: %lf ms\n", (parallel_end - parallel_start)*1000);
}

int main()
{
	printf("==========PARALLEL IMPLEMENTATION OF DIJKSTRA ALGORITHM==========\n");
	printf("Enter number of vertices: ");
	scanf("%d",&V);
	printf("Enter number of edges: ");
	scanf("%d",&E);
	Vertex vertices[V];
	Edge edges[E];
	int weights[E];

	int i;
	for(i = 0; i < V; i++)
	{
		Vertex a = { .label =i , .visited=false};
		vertices[i] = a;
	}

	printf("\nEnter these details \nFROM \tTO \tWEIGHT\n");
	int from,to,weight;
	for(i = 0; i < E; i++)
	{
		scanf("%d %d %d",&from,&to,&weight);
		Edge e = {.u = from , .v = to};
		edges[i] = e;
		weights[i] = weight;
	}

	int source;
	printf("\nEnter Source Vertex: ");
	scanf("%d",&source);
	Vertex root = {source, false};

	Dijkstra_Parallel(vertices, edges, weights, &root);

	return 0;
}

Overwriting codes.cpp


In [49]:
!g++ -fopenmp codes.cpp

In [50]:
!./a.out

==========PARALLEL IMPLEMENTATION OF DIJKSTRA ALGORITHM==========
Enter number of vertices: 6
Enter number of edges: 8

Enter these details 
FROM 	TO 	WEIGHT
0 1 40
0 2 15
1 2 20
1 3 10
1 4 25
2 3 100
1 5 6
4 5 8

Enter Source Vertex: 0

VERTEX 	SHORTEST PATH LENGTH 
0 	0
1 	40
2 	15
3 	50
4 	65
5 	46

Running time: 0.154631 ms


In [ ]:
66